# A Reduction Tale

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 42.395 MiB


and choose a different container for the data that we want to reduce, starting with a list:

## Regular lists

In [2]:
a = [float(i) for i in range(10*1000*1000)]

In [2] used 312.688 MiB RAM in 1.963s, peaked 0.000 MiB above current, total RAM usage 355.082 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

10 loops, best of 3: 52 ms per loop
In [3] used -1.723 MiB RAM in 2.232s, peaked 0.000 MiB above current, total RAM usage 353.359 MiB


which, in MFLOPS (Mega-FloatingPointOps-Per-Second) is:

In [4]:
print("MFLOPS:", round((len(a) / t.best / 1e6), 1))

MFLOPS: 192.3
In [4] used 0.000 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 353.359 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

## NumPy

In [5]:
import numpy as np

In [5] used 12.570 MiB RAM in 0.179s, peaked 0.000 MiB above current, total RAM usage 365.930 MiB


In [6]:
na = np.array(a, dtype=np.float64)

In [6] used 76.176 MiB RAM in 0.392s, peaked 0.000 MiB above current, total RAM usage 442.105 MiB


In [7]:
print("SIZE:", round((na.size * na.itemsize) / 2**20., 3))

SIZE: 76.294
In [7] used 0.000 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 442.105 MiB


We see that, with 8 bytes/element, NumPy is a very efficient container.

In [8]:
t = %timeit -o sum(na)

1 loop, best of 3: 943 ms per loop
In [8] used 0.250 MiB RAM in 3.882s, peaked 0.000 MiB above current, total RAM usage 442.355 MiB


In [9]:
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

MFLOPS: 10.608
In [9] used 0.000 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 442.355 MiB


### Exercise

The performance for NumPy is several times slower than the computation with the list.  Why so?

*Hint: * We are using sum() which is a Python function.

### Solution

NumPy has a lot of overhead in producing a Python integer for every element in the array for feeding it to the sum().

*Hint:* Use internal NumPy methods (ufuncs) when possible.

In [10]:
t = %timeit -o na.sum()

100 loops, best of 3: 4.38 ms per loop
In [10] used 0.000 MiB RAM in 1.859s, peaked 0.000 MiB above current, total RAM usage 442.355 MiB


In [11]:
print("FLOPS:", round(len(a) / t.best / 1e6, 3))

FLOPS: 2281.905
In [11] used 0.000 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 442.355 MiB


This is more than 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

## Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum memory speed that supports your laptop?

#### Solution

In [12]:
# This is an easy one.  Just divide the size of the dataset by the time that takes the reduction
(na.size * na.itemsize) / t.best / 2**30

17.00151538116094

In [12] used 0.000 MiB RAM in 0.019s, peaked 0.000 MiB above current, total RAM usage 442.355 MiB


So, in this case the memory bandwidth is ~ 17 GB/s.

## Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [46]:
import bcolz
bcolz.print_versions()
bcolz.defaults.cparams['cname'] = 'blosclz'
bcolz.defaults.cparams['clevel'] = 9
bcolz.defaults.cparams['shuffle'] = bcolz.SHUFFLE
bcolz.set_nthreads(4)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
bcolz version:     1.1.1.dev20
bcolz git info:    b'1.1.0-20-g50a83ca'
NumPy version:     1.11.0
Blosc version:     1.9.3 ($Date:: 2016-07-06 #$)
Blosc compressors: ['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib']
Numexpr version:   2.5.2
Dask version:      0.9.0
Python version:    3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Platform:          linux-x86_64
Byte-ordering:     little
Detected cores:    4
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


4

In [46] used 0.004 MiB RAM in 0.018s, peaked 0.000 MiB above current, total RAM usage 647.863 MiB


In [76]:
ca = bcolz.carray(na)

In [76] used 0.000 MiB RAM in 0.044s, peaked 0.000 MiB above current, total RAM usage 741.758 MiB


In [77]:
print("mem_used:", mw.measurements.memory_delta)

mem_used: 0.0
In [77] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 741.758 MiB


Ok, this time the amount of memory used seems much lower.  Also, bcolz containers can provide an estimation on how much memory they are taking; let's have a look:

In [78]:
ca

carray((10000000,), float64)
  nbytes := 76.29 MB; cbytes := 1018.33 KB; ratio: 76.72
  cparams := cparams(clevel=9, shuffle=1, cname='blosclz', quantize=0)
  chunklen := 65536; chunksize: 524288; blocksize: 524288
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [78] used 0.000 MiB RAM in 0.007s, peaked 0.000 MiB above current, total RAM usage 741.758 MiB


In this case we see that bcolz estimation is reasonably close to `ipython_memwatcher` measurements.  Let's have a look at the speed of the reduction:

In [79]:
t = %timeit -o ca.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

10 loops, best of 3: 20 ms per loop
MFLOPS: 500.668
In [79] used 0.000 MiB RAM in 0.906s, peaked 0.000 MiB above current, total RAM usage 741.758 MiB


This is around 2~5x slower (depending on the machine) than a regular NumPy array, but the size of the array is an impressive 76x smaller.  But is compression the only responsible of the overhead?  Let's investigate a bit further.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [51]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [51] used 76.250 MiB RAM in 0.428s, peaked 0.266 MiB above current, total RAM usage 725.949 MiB


In [52]:
cau

carray((10000000,), float64)
  nbytes := 76.29 MB; cbytes := 76.50 MB; ratio: 1.00
  cparams := cparams(clevel=0, shuffle=1, cname='blosclz', quantize=0)
  chunklen := 65536; chunksize: 524288; blocksize: 8192
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [52] used 0.000 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 725.949 MiB


In [80]:
t = %timeit -o cau.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

100 loops, best of 3: 12.5 ms per loop
MFLOPS: 801.816
In [80] used 0.000 MiB RAM in 5.271s, peaked 0.000 MiB above current, total RAM usage 741.758 MiB


As we can see, the times with an uncompressed `carray` are between 1.5x and 2x faster than with a compressed one, so compressing is not the only source of the overhead.  The other source of the difference is the memory layout of the different containers (bcolz's carray data container layout is a bit more complex than NumPy).

So, while bcolz allows to use compressed in-memory data containers, this usually represents more cost in performance (compared with NumPy).  But sometimes you may prefer to keep more data in-memory and assume that the computations are going to be slower.

## Exercise

bcolz uses Blosc, a multithreaded meta-compressor, to do the compression under the hood.  Blosc can use different codecs, and each one has different behavior in terms of performance.  Given the next computation:

In [58]:
bcolz.defaults.cparams['cname'] = 'blosclz'
bcolz.defaults.cparams['clevel'] = 9
bcolz.defaults.cparams['shuffle'] = bcolz.SHUFFLE
bcolz.set_nthreads(4)
ca = bcolz.carray(na)
%timeit ca.sum()

10 loops, best of 3: 20.1 ms per loop
In [58] used 0.000 MiB RAM in 0.936s, peaked 0.000 MiB above current, total RAM usage 741.566 MiB


Play with the different parameters and see:

1) Which provides the best compression

2) Which the fastest speed

3) The combination that strikes a good balance between compression and performance